In [2]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/28 20:07:04 WARN Utils: Your hostname, Vinoths-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
22/02/28 20:07:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 20:07:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Question 1. Install Spark and PySpark and Check the Spark Version
spark.version

'3.2.1'

In [18]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-28 20:10:30--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.79.180
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.79.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv.1’

fhvhv_tripdata_2021 100%[===================>] 699.83M  20.4MB/s    in 40s     

2022-02-28 20:11:10 (17.6 MB/s) - ‘fhvhv_tripdata_2021-02.csv.1’ saved [733822658/733822658]



In [19]:
!wc -l fhvhv_tripdata_2021-02.csv

 11613943 fhvhv_tripdata_2021-02.csv


In [20]:
from pyspark.sql import types

In [21]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [22]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [23]:
df = df.repartition(24).write.parquet('fhvhv/2021/02/', mode='overwrite')

22/02/28 20:11:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/28 20:11:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/28 20:11:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/28 20:11:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [26]:
# df.count()
# Question 2. HVFHW February 2021 - What's the size of the folder with results (in MB)?
!du -sh fhvhv/

212M	fhvhv/


In [30]:
df = spark.read.parquet('fhvhv/2021/02/')
df.createOrReplaceTempView('fhvhv_feb2021')

In [34]:
# Question 3 - Count the records: How many taxi trips were there on February 15?

spark.sql("""
SELECT COUNT(1) FROM fhvhv_feb2021 WHERE DATE(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [44]:
# Question 4. Longest trip for each day
tdf = spark.sql("""
SELECT a.*,
(unix_timestamp(a.dropoff_datetime) - unix_timestamp(a.pickup_datetime)) AS duration_in_seconds
FROM fhvhv_feb2021 a
ORDER BY duration_in_seconds DESC LIMIT 1
""")
tdf.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|duration_in_seconds|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|              75540|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------------+



In [48]:
# Question 5. Most frequent dispatching_base_num - How many stages this spark job has?
tdf2=spark.sql("""
SELECT
    dispatching_base_num,  COUNT(dispatching_base_num) as volume
FROM fhvhv_feb2021
GROUP BY dispatching_base_num
ORDER BY volume DESC
""")
tdf2.show()

+--------------------+-------+
|dispatching_base_num| volume|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [49]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-28 22:13:52--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.194.160
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.194.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-28 22:13:52 (35.0 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [53]:
df_zones = spark.read.csv('taxi+_zone_lookup.csv',header=True)
df_zones.createOrReplaceTempView('zone_lookup')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [64]:
# Question 6. Most common locations pair
tdf3 = spark.sql("""
select concat (coalesce(pz.Zone,'Unknown'),' / ', coalesce(dz.Zone,'Unknown')) as pickup_dropoff_pair, count(1) cnt
from fhvhv_feb2021 t, zone_lookup pz, zone_lookup dz 
where t.PULocationID = pz.LocationID and
t.DOLocationID = dz.LocationID
group by pickup_dropoff_pair
order by cnt desc limit 1
""")
tdf3.show(truncate=False)

+-----------------------------+-----+
|pickup_dropoff_pair          |cnt  |
+-----------------------------+-----+
|East New York / East New York|45041|
+-----------------------------+-----+

